# One-Time Setup: Mount ADLS Gen2 Storage

This notebook creates persistent mount points to our ADLS Gen2 containers.

**You only need to run this code once for the entire workspace.** The mounts will remain available until they are explicitly unmounted.

In [0]:
# TO FILL IN
TENANT_ID = ""
CLIENT_ID = ""
CLIENT_SECRET = "" 
STORAGE_ACCOUNT_NAME = "" 

configs = {
  "fs.azure.account.auth.type": "OAuth",
  "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
  "fs.azure.account.oauth2.client.id": CLIENT_ID,
  "fs.azure.account.oauth2.client.secret": CLIENT_SECRET,
  "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{TENANT_ID}/oauth2/token"
}

# Mount the bronze, silver, and gold containers
for container in ["bronze", "silver", "gold"]:
  mount_point = f"/mnt/{container}"
  source_uri = f"abfss://{container}@{STORAGE_ACCOUNT_NAME}.dfs.core.windows.net/"
  
  try:
    if any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
      print(f"Unmounting existing mount point: {mount_point}")
      dbutils.fs.unmount(mount_point)
  except Exception as e:
    print(f"Error unmounting {mount_point}: {e}")
    
  try:
    print(f"Mounting {source_uri} to {mount_point}...")
    dbutils.fs.mount(
      source = source_uri,
      mount_point = mount_point,
      extra_configs = configs
    )
    print(f"Successfully mounted {mount_point}")
  except Exception as e:
    print(f"!!! ERROR mounting {mount_point}: {e}")